# Recherche des catalogues

In [1]:
%matplotlib inline

import os

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
import pyarrow

In [3]:
# Verify SparkContext
print(sc)

# Print Spark version
print(sc.version)

<SparkContext master=mesos://vm-75063.lal.in2p3.fr:5050 appName=PySparkShell>
2.4.4


In [4]:
from pyspark.sql.functions import col, pandas_udf 
from pyspark.sql.types import LongType

from pyspark.sql.types import IntegerType,FloatType
from pyspark.sql.functions import pandas_udf, PandasUDFType

In [5]:
# here is how we create a function ('Ang2Pix') that can be called by dataframes
# it takes as input the "ra" and "dec" values (which are not very different from theta/phi)
# and returns the pixel number (but as pandas series for efficiency)
import numpy as np
import pandas as pd
import healpy as hp

nside=512

In [6]:
! hadoop dfs -ls /lsst/DC2/cosmoDC2

DEPRECATED: Use of this script to execute hdfs command is deprecated.
Instead use the hdfs command for it.

Found 5 items
drwxr-xr-x   - stephane.plaszczynski lsst          0 2019-09-03 20:04 /lsst/DC2/cosmoDC2/cosmoDC2_v1.1.4_image.parquet
drwxr-xr-x   - stephane.plaszczynski lsst          0 2019-10-16 10:52 /lsst/DC2/cosmoDC2/cosmoDC2_v1.1.4_image_double.parquet
drwxr-xr-x   - stephane.plaszczynski lsst          0 2020-04-23 17:28 /lsst/DC2/cosmoDC2/cosmoDC2_v1.1.4_image_nofaint.parquet
drwxr-xr-x   - stephane.plaszczynski lsst          0 2020-09-07 19:17 /lsst/DC2/cosmoDC2/shear_cosmoDC2
drwxr-xr-x   - stephane.plaszczynski lsst          0 2019-04-16 18:40 /lsst/DC2/cosmoDC2/xyz_v1.1.4_hive


In [7]:
! hadoop dfs -ls /lsst/DC2/cosmoDC2/shear_cosmoDC2/10066/

DEPRECATED: Use of this script to execute hdfs command is deprecated.
Instead use the hdfs command for it.

-rw-r--r--   3 stephane.plaszczynski lsst  240603132 2020-09-07 19:14 /lsst/DC2/cosmoDC2/shear_cosmoDC2/10066


In [8]:
base_dir="/lsst/DC2/cosmoDC2/shear_cosmoDC2"

In [9]:
fullfilename="/lsst/DC2/cosmoDC2/cosmoDC2_v1.1.4_image.parquet"
datafile=fullfilename
FLAG_ONE_FILE_INPUT=True

In [10]:
#from subprocess import Popen, PIPE
#hdfs_path = base_dir
#process = Popen(f'hdfs dfs -ls -h {hdfs_path}', shell=True, stdout=PIPE, stderr=PIPE)
#std_out, std_err = process.communicate()
#list_of_file_names = [fn.split(' ')[-1].split('/')[-1] for fn in std_out.decode().readlines()[1:]][:-1]
#list_of_file_names_with_full_address = [fn.split(' ')[-1] for fn in std_out.decode().readlines()[1:]][:-1]

In [11]:
#for fn in std_out.readlines():
#    print(fn)

In [12]:
from pyspark.sql import SparkSession

# Initialise our Spark session
spark = SparkSession.builder.getOrCreate()

In [13]:
# Read the data as DataFrame
if FLAG_ONE_FILE_INPUT:
    df = spark.read.format("parquet").load(datafile)
else: # read all files
    #the_list_of_files=os.listdir(base_dir)
    #the_sorted_list_of_files=sorted(the_list_of_files)
    #print("sorted files : ",the_sorted_list_of_files[:5])
    #selected_files=[]
    #for filename in the_sorted_list_of_files:
    #    sel_filename=re.findall("^dc2_object_run2.2i_dr6c_tract[0-9][0-9][0-9][0-9].parquet$",filename)
    #    if len(sel_filename) > 0:
    #        selected_files.append(sel_filename[0])
    #print("selected files : ", selected_files[:5])
    #InputPath = [base_dir + "/{}".format(filename) for filename in selected_files]
    InputPath = all_files  # filename from the decoded index file
    df = spark.read.parquet(*InputPath)        

In [14]:
df.printSchema()

root
 |-- Mag_true_g_lsst_z0: float (nullable = true)
 |-- Mag_true_z_lsst_z0: float (nullable = true)
 |-- ra: float (nullable = true)
 |-- mag_y: float (nullable = true)
 |-- mag_z: float (nullable = true)
 |-- size_true: float (nullable = true)
 |-- halo_id: long (nullable = true)
 |-- position_x: float (nullable = true)
 |-- mag_g: float (nullable = true)
 |-- Mag_true_u_lsst_z0: float (nullable = true)
 |-- mag_true_r: float (nullable = true)
 |-- position_angle_true: float (nullable = true)
 |-- mag_true_g: float (nullable = true)
 |-- stellar_mass: float (nullable = true)
 |-- Mag_true_y_lsst_z0: float (nullable = true)
 |-- position_y: float (nullable = true)
 |-- mag_true_i: float (nullable = true)
 |-- redshift: float (nullable = true)
 |-- Mag_true_i_lsst_z0: float (nullable = true)
 |-- mag_r: float (nullable = true)
 |-- mag_true_y: float (nullable = true)
 |-- dec: float (nullable = true)
 |-- mag_true_u: float (nullable = true)
 |-- Mag_true_r_lsst_z0: float (nullable = 

In [15]:
rs=df.select("redshift")

In [16]:
rs.show()

+------------+
|    redshift|
+------------+
|  0.02164939|
|  0.02191215|
| 0.021491187|
| 0.021335145|
| 0.006085375|
|0.0062329015|
| 0.022323765|
| 0.013605996|
| 0.020426989|
| 0.011232678|
| 0.007916852|
| 0.018281268|
| 0.015733609|
| 0.021410722|
| 0.021449156|
|  0.02179927|
| 0.020016473|
| 0.017602554|
| 0.019653492|
| 0.021616712|
+------------+
only showing top 20 rows



In [17]:
dp_rs=rs.toPandas()
dp_rs.info()

Py4JJavaError: An error occurred while calling o65.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 4.0 failed 4 times, most recent failure: Lost task 7.3 in stage 4.0 (TID 913, vm-75171.lal.in2p3.fr, executor 1): org.apache.spark.SparkException: Kryo serialization failed: Buffer overflow. Available: 0, required: 19675535. To avoid this, increase spark.kryoserializer.buffer.max value.
	at org.apache.spark.serializer.KryoSerializerInstance.serialize(KryoSerializer.scala:354)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:456)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Caused by: com.esotericsoftware.kryo.KryoException: Buffer overflow. Available: 0, required: 19675535
	at com.esotericsoftware.kryo.io.Output.require(Output.java:167)
	at com.esoterics